# Import Libraries

In [24]:
from urllib import request, parse
import pandas as pd

# Get Data From Socrata

SoQL document can be found [here](https://dev.socrata.com/docs/functions/)

In [25]:
def get_from_socrata(endpoint, sql='', fname=None):
    print()
    (fpath, http_msg) = request.urlretrieve(endpoint+'?$query='+parse.quote(sql), filename=fname)
    print("File Saved as ", fpath)
    print("HTTP Message retrieved as below:", '\n', http_msg)

In [33]:
# get the automated traffic count dataset
endpoint = 'https://data.cityofnewyork.us/resource/7ym2-wayt.csv'
sql = 'select segmentid, street, direction, count(mm) \
        where yr = 2015 \
        group by segmentid, street, direction \
        order by segmentid \
        limit 1000'
get_from_socrata(endpoint, sql=sql, fname="traffic-data/traffic_auto_1.csv")        


File Saved as  traffic-data/traffic_auto_1.csv
HTTP Message retrieved as below: 
 Server: nginx
Date: Sun, 23 Jul 2023 13:36:48 GMT
Content-Type: text/csv; charset=UTF-8
Transfer-Encoding: chunked
Connection: close
Access-Control-Allow-Origin: *
ETag: W/"YnJhdm8uNDIyNjA5XzNfMzVlU1pOSzBiZThiZDZkRGRLVXg2cGt6ZjUxTTA---gziq_OcxcM2csPGbivd6bRsZA4Rinlw--gzip--gzip"
X-SODA2-Fields: ["segmentid","street","direction","count_mm"]
X-SODA2-Types: ["number","text","text","number"]
X-SODA2-Data-Out-Of-Date: false
X-SODA2-Truth-Last-Modified: Thu, 29 Dec 2022 21:13:19 GMT
X-SODA2-Secondary-Last-Modified: Thu, 29 Dec 2022 21:13:19 GMT
Last-Modified: Thu, 29 Dec 2022 21:13:19 GMT
Vary: Accept-Encoding
Age: 22
X-Socrata-Region: aws-us-east-1-fedramp-prod
Strict-Transport-Security: max-age=31536000; includeSubDomains
X-Socrata-RequestId: 231a5d57804cae8034f5ff2859000127




In [34]:
# get the AADT dataset
endpoint = "https://data.ny.gov/resource/6amx-2pbv.csv"
sql = 'select * where aadt_year = "2015"'
get_from_socrata(endpoint, sql = sql, fname = "traffic-data/aadt_1.csv")


File Saved as  traffic-data/aadt_1.csv
HTTP Message retrieved as below: 
 Server: nginx
Date: Sun, 23 Jul 2023 13:37:13 GMT
Content-Type: text/csv; charset=UTF-8
Transfer-Encoding: chunked
Connection: close
Access-Control-Allow-Origin: *
ETag: W/"YWxwaGEuODE0OV81XzExNmFsdG9TQ2hLamZDRTJsQV9pWnJTZEpsTUdjWQ---gzioWST2C2uK75E1vRKSgxCtfNC7aP--gzip--gzip"
X-SODA2-Fields: ["aadt_year","station_id","county","signing","state_route","county_road","road_name","beginning_description","ending_description","municipality","length","fc","ramp","bridge","rr_xing","oneway","count"]
X-SODA2-Types: ["text","number","text","text","text","text","text","text","text","text","number","number","text","text","text","text","number"]
X-SODA2-Data-Out-Of-Date: false
X-SODA2-Truth-Last-Modified: Tue, 04 Feb 2020 22:07:45 GMT
X-SODA2-Secondary-Last-Modified: Tue, 04 Feb 2020 22:07:45 GMT
Last-Modified: Tue, 04 Feb 2020 22:07:45 GMT
Vary: Accept-Encoding
Age: 0
X-Socrata-Region: aws-us-east-1-fedramp-prod
Strict-Tran

In [19]:
print(http_msg)

None


# Import  Data

In [7]:
traffic = pd.read_csv("traffic.csv")
print(traffic.info())
print(traffic.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   requestid  10000 non-null  int64 
 1   boro       10000 non-null  object
 2   yr         10000 non-null  int64 
 3   m          10000 non-null  int64 
 4   d          10000 non-null  int64 
 5   hh         10000 non-null  int64 
 6   mm         10000 non-null  int64 
 7   vol        10000 non-null  int64 
 8   segmentid  10000 non-null  int64 
 9   wktgeom    10000 non-null  object
 10  street     10000 non-null  object
 11  fromst     10000 non-null  object
 12  tost       10000 non-null  object
 13  direction  10000 non-null  object
dtypes: int64(8), object(6)
memory usage: 1.1+ MB
None
   requestid    boro    yr  m  d  hh  mm  vol  segmentid   
0      17976  Queens  2015  2  7  10  30   28     148877  \
1      17976  Queens  2015  2  7  10  45   46     148877   
2      17976  Queens  2015  2  7 